In [1]:
import gym
import pandas as pd
import numpy as np
from gym import envs
import random
import scipy.stats


In [5]:

class LinearSoftmaxAgent(object):
    """Act with softmax policy. Features are encoded as
    phi(s, a) is a 1-hot vector of states."""
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.states = []
        self.actions = []
        self.probs = []
        self.rewards = []
        self.theta = np.random.random(state_size * action_size)
        self.alpha = .01
        self.gamma = .99

    def store(self, state, action, prob, reward):
        self.states.append(state)
        self.actions.append(action)
        self.probs.append(prob)
        self.rewards.append(reward)

    def _phi(self, s, a):
        encoded = np.zeros([self.action_size, self.state_size])
        encoded[a] = s
        return encoded.flatten()

    def _softmax(self, s, a):
        return np.exp(self.theta.dot(self._phi(s, a)) / 100)

    def pi(self, s):
        """\pi(a | s)"""
        weights = np.empty(self.action_size)
        for a in range(self.action_size):
            weights[a] = self._softmax(s, a)
        return weights / np.sum(weights)

    def act(self, state):
        probs = self.pi(state)
        a = random.choices(range(0, self.action_size), weights=probs)
        a = a[0]
        pi = probs[a]
        return (a, pi)

    def _gradient(self, s, a):
        expected = 0
        probs = self.pi(s)
        for b in range(0, self.action_size):
            expected += probs[b] * self._phi(s, b)
        return self._phi(s, a) - expected

    def _R(self, t):
        """Reward function."""
        total = 0
        for tau in range(t, len(self.rewards)):
            total += self.gamma**(tau - t) * self.rewards[tau]
        return total

    def train(self):
        self.rewards -= np.mean(self.rewards)
        self.rewards /= np.std(self.rewards)
        for t in range(len(self.states)):
            s = self.states[t]
            a = self.actions[t]
            r = self._R(t)
            grad = self._gradient(s, a)
            self.theta = self.theta + self.alpha * r * grad
        # print(self.theta)
        self.states = []
        self.actions = []
        self.probs = []
        self.rewards = []

    def getName(self):
        return 'LinearSoftmaxAgent'

    def save(self):
        pass

In [6]:
import gym

SAVE_FREQUENCY = 10
env = gym.make('CartPole-v1')
state = env.reset()
score = 0
episode = 0
prev_frame = None
state_size = 4
action_size = env.action_space.n
g = LinearSoftmaxAgent(state_size, action_size)


MAX_EPISODES = 10000
while episode < MAX_EPISODES:  # episode loop
    # env.render()
    action, prob = g.act(state)
    state, reward, done, info = env.step(action)  # take a random action
    if done:
        reward = -10
    score += reward
    g.store(state, action, prob, reward)

    if done:
        episode += 1
        g.train()
        print('Episode: {} Score: {}'.format(episode, score))
        score = 0
        state = env.reset()
        if episode % SAVE_FREQUENCY == 0:
            g.save()

Episode: 1 Score: 15.0
Episode: 2 Score: 27.0
Episode: 3 Score: 1.0
Episode: 4 Score: -1.0
Episode: 5 Score: 18.0
Episode: 6 Score: 5.0
Episode: 7 Score: 50.0
Episode: 8 Score: 14.0
Episode: 9 Score: 0.0
Episode: 10 Score: 30.0
Episode: 11 Score: 18.0
Episode: 12 Score: 4.0
Episode: 13 Score: 9.0
Episode: 14 Score: 7.0
Episode: 15 Score: 9.0
Episode: 16 Score: 13.0
Episode: 17 Score: 0.0
Episode: 18 Score: 19.0
Episode: 19 Score: 6.0
Episode: 20 Score: 6.0
Episode: 21 Score: 2.0
Episode: 22 Score: 3.0
Episode: 23 Score: 27.0
Episode: 24 Score: 3.0
Episode: 25 Score: 13.0
Episode: 26 Score: 23.0
Episode: 27 Score: 2.0
Episode: 28 Score: -2.0
Episode: 29 Score: 20.0
Episode: 30 Score: 81.0
Episode: 31 Score: 8.0
Episode: 32 Score: 15.0
Episode: 33 Score: 16.0
Episode: 34 Score: 14.0
Episode: 35 Score: 8.0
Episode: 36 Score: 7.0
Episode: 37 Score: 10.0
Episode: 38 Score: 11.0
Episode: 39 Score: 10.0
Episode: 40 Score: 6.0
Episode: 41 Score: 5.0
Episode: 42 Score: 32.0
Episode: 43 Score: 3

Episode: 370 Score: 88.0
Episode: 371 Score: 19.0
Episode: 372 Score: 79.0
Episode: 373 Score: 9.0
Episode: 374 Score: 8.0
Episode: 375 Score: 28.0
Episode: 376 Score: 39.0
Episode: 377 Score: 53.0
Episode: 378 Score: 51.0
Episode: 379 Score: 39.0
Episode: 380 Score: 11.0
Episode: 381 Score: 20.0
Episode: 382 Score: 81.0
Episode: 383 Score: 24.0
Episode: 384 Score: 23.0
Episode: 385 Score: 21.0
Episode: 386 Score: 42.0
Episode: 387 Score: 45.0
Episode: 388 Score: 35.0
Episode: 389 Score: 30.0
Episode: 390 Score: 19.0
Episode: 391 Score: 36.0
Episode: 392 Score: 55.0
Episode: 393 Score: 45.0
Episode: 394 Score: 57.0
Episode: 395 Score: 16.0
Episode: 396 Score: 83.0
Episode: 397 Score: 17.0
Episode: 398 Score: 14.0
Episode: 399 Score: 26.0
Episode: 400 Score: 18.0
Episode: 401 Score: 32.0
Episode: 402 Score: 9.0
Episode: 403 Score: 33.0
Episode: 404 Score: 4.0
Episode: 405 Score: 12.0
Episode: 406 Score: 34.0
Episode: 407 Score: 60.0
Episode: 408 Score: 12.0
Episode: 409 Score: 6.0
Episo

Episode: 713 Score: 65.0
Episode: 714 Score: 48.0
Episode: 715 Score: 30.0
Episode: 716 Score: 33.0
Episode: 717 Score: 12.0
Episode: 718 Score: 34.0
Episode: 719 Score: 39.0
Episode: 720 Score: 26.0
Episode: 721 Score: 62.0
Episode: 722 Score: 15.0
Episode: 723 Score: 60.0
Episode: 724 Score: 17.0
Episode: 725 Score: 23.0
Episode: 726 Score: 28.0
Episode: 727 Score: 21.0
Episode: 728 Score: 56.0
Episode: 729 Score: 18.0
Episode: 730 Score: 10.0
Episode: 731 Score: 57.0
Episode: 732 Score: 68.0
Episode: 733 Score: 34.0
Episode: 734 Score: 29.0
Episode: 735 Score: 44.0
Episode: 736 Score: 28.0
Episode: 737 Score: 24.0
Episode: 738 Score: 38.0
Episode: 739 Score: 32.0
Episode: 740 Score: 22.0
Episode: 741 Score: 50.0
Episode: 742 Score: 9.0
Episode: 743 Score: 26.0
Episode: 744 Score: 31.0
Episode: 745 Score: 37.0
Episode: 746 Score: 29.0
Episode: 747 Score: 48.0
Episode: 748 Score: 63.0
Episode: 749 Score: 86.0
Episode: 750 Score: 21.0
Episode: 751 Score: 34.0
Episode: 752 Score: 19.0
E

Episode: 1045 Score: 85.0
Episode: 1046 Score: 67.0
Episode: 1047 Score: 55.0
Episode: 1048 Score: 35.0
Episode: 1049 Score: 63.0
Episode: 1050 Score: 190.0
Episode: 1051 Score: 49.0
Episode: 1052 Score: 56.0
Episode: 1053 Score: 23.0
Episode: 1054 Score: 53.0
Episode: 1055 Score: 31.0
Episode: 1056 Score: 47.0
Episode: 1057 Score: 28.0
Episode: 1058 Score: 20.0
Episode: 1059 Score: 81.0
Episode: 1060 Score: 85.0
Episode: 1061 Score: 58.0
Episode: 1062 Score: 54.0
Episode: 1063 Score: 62.0
Episode: 1064 Score: 35.0
Episode: 1065 Score: 52.0
Episode: 1066 Score: 47.0
Episode: 1067 Score: 74.0
Episode: 1068 Score: 40.0
Episode: 1069 Score: 31.0
Episode: 1070 Score: 25.0
Episode: 1071 Score: 67.0
Episode: 1072 Score: 31.0
Episode: 1073 Score: 42.0
Episode: 1074 Score: 26.0
Episode: 1075 Score: 95.0
Episode: 1076 Score: 53.0
Episode: 1077 Score: 62.0
Episode: 1078 Score: 101.0
Episode: 1079 Score: 70.0
Episode: 1080 Score: 41.0
Episode: 1081 Score: 45.0
Episode: 1082 Score: 56.0
Episode: 1

Episode: 1362 Score: 108.0
Episode: 1363 Score: 60.0
Episode: 1364 Score: 32.0
Episode: 1365 Score: 38.0
Episode: 1366 Score: 71.0
Episode: 1367 Score: 26.0
Episode: 1368 Score: 85.0
Episode: 1369 Score: 145.0
Episode: 1370 Score: 49.0
Episode: 1371 Score: 69.0
Episode: 1372 Score: 55.0
Episode: 1373 Score: 33.0
Episode: 1374 Score: 31.0
Episode: 1375 Score: 128.0
Episode: 1376 Score: 49.0
Episode: 1377 Score: 129.0
Episode: 1378 Score: 52.0
Episode: 1379 Score: 52.0
Episode: 1380 Score: 44.0
Episode: 1381 Score: 35.0
Episode: 1382 Score: 47.0
Episode: 1383 Score: 99.0
Episode: 1384 Score: 23.0
Episode: 1385 Score: 38.0
Episode: 1386 Score: 44.0
Episode: 1387 Score: 80.0
Episode: 1388 Score: 48.0
Episode: 1389 Score: 33.0
Episode: 1390 Score: 49.0
Episode: 1391 Score: 36.0
Episode: 1392 Score: 46.0
Episode: 1393 Score: 59.0
Episode: 1394 Score: 107.0
Episode: 1395 Score: 42.0
Episode: 1396 Score: 29.0
Episode: 1397 Score: 74.0
Episode: 1398 Score: 48.0
Episode: 1399 Score: 42.0
Episode

...

Episode: 8560 Score: 57.0
Episode: 8561 Score: 80.0
Episode: 8562 Score: 51.0
Episode: 8563 Score: 47.0
Episode: 8564 Score: 50.0
Episode: 8565 Score: 42.0
Episode: 8566 Score: 47.0
Episode: 8567 Score: 106.0
Episode: 8568 Score: 58.0
Episode: 8569 Score: 77.0
Episode: 8570 Score: 73.0
Episode: 8571 Score: 109.0
Episode: 8572 Score: 61.0
Episode: 8573 Score: 215.0
Episode: 8574 Score: 71.0
Episode: 8575 Score: 35.0
Episode: 8576 Score: 32.0
Episode: 8577 Score: 43.0
Episode: 8578 Score: 38.0
Episode: 8579 Score: 99.0
Episode: 8580 Score: 61.0
Episode: 8581 Score: 175.0
Episode: 8582 Score: 137.0
Episode: 8583 Score: 45.0
Episode: 8584 Score: 49.0
Episode: 8585 Score: 95.0
Episode: 8586 Score: 68.0
Episode: 8587 Score: 41.0
Episode: 8588 Score: 55.0
Episode: 8589 Score: 58.0
Episode: 8590 Score: 29.0
Episode: 8591 Score: 53.0
Episode: 8592 Score: 135.0
Episode: 8593 Score: 75.0
Episode: 8594 Score: 73.0
Episode: 8595 Score: 51.0
Episode: 8596 Score: 66.0
Episode: 8597 Score: 105.0
Episo

Episode: 8874 Score: 133.0
Episode: 8875 Score: 76.0
Episode: 8876 Score: 69.0
Episode: 8877 Score: 154.0
Episode: 8878 Score: 80.0
Episode: 8879 Score: 167.0
Episode: 8880 Score: 189.0
Episode: 8881 Score: 73.0
Episode: 8882 Score: 101.0
Episode: 8883 Score: 72.0
Episode: 8884 Score: 103.0
Episode: 8885 Score: 44.0
Episode: 8886 Score: 51.0
Episode: 8887 Score: 66.0
Episode: 8888 Score: 55.0
Episode: 8889 Score: 91.0
Episode: 8890 Score: 85.0
Episode: 8891 Score: 34.0
Episode: 8892 Score: 61.0
Episode: 8893 Score: 88.0
Episode: 8894 Score: 62.0
Episode: 8895 Score: 41.0
Episode: 8896 Score: 67.0
Episode: 8897 Score: 192.0
Episode: 8898 Score: 107.0
Episode: 8899 Score: 51.0
Episode: 8900 Score: 42.0
Episode: 8901 Score: 66.0
Episode: 8902 Score: 43.0
Episode: 8903 Score: 188.0
Episode: 8904 Score: 76.0
Episode: 8905 Score: 81.0
Episode: 8906 Score: 153.0
Episode: 8907 Score: 46.0
Episode: 8908 Score: 61.0
Episode: 8909 Score: 91.0
Episode: 8910 Score: 53.0
Episode: 8911 Score: 78.0
Ep

Episode: 9196 Score: 103.0
Episode: 9197 Score: 42.0
Episode: 9198 Score: 113.0
Episode: 9199 Score: 57.0
Episode: 9200 Score: 69.0
Episode: 9201 Score: 39.0
Episode: 9202 Score: 87.0
Episode: 9203 Score: 53.0
Episode: 9204 Score: 89.0
Episode: 9205 Score: 157.0
Episode: 9206 Score: 39.0
Episode: 9207 Score: 61.0
Episode: 9208 Score: 85.0
Episode: 9209 Score: 65.0
Episode: 9210 Score: 65.0
Episode: 9211 Score: 46.0
Episode: 9212 Score: 164.0
Episode: 9213 Score: 42.0
Episode: 9214 Score: 150.0
Episode: 9215 Score: 102.0
Episode: 9216 Score: 39.0
Episode: 9217 Score: 53.0
Episode: 9218 Score: 63.0
Episode: 9219 Score: 91.0
Episode: 9220 Score: 114.0
Episode: 9221 Score: 76.0
Episode: 9222 Score: 108.0
Episode: 9223 Score: 71.0
Episode: 9224 Score: 39.0
Episode: 9225 Score: 48.0
Episode: 9226 Score: 82.0
Episode: 9227 Score: 59.0
Episode: 9228 Score: 59.0
Episode: 9229 Score: 56.0
Episode: 9230 Score: 39.0
Episode: 9231 Score: 72.0
Episode: 9232 Score: 77.0
Episode: 9233 Score: 249.0
Epi

Episode: 9514 Score: 183.0
Episode: 9515 Score: 121.0
Episode: 9516 Score: 51.0
Episode: 9517 Score: 101.0
Episode: 9518 Score: 80.0
Episode: 9519 Score: 93.0
Episode: 9520 Score: 54.0
Episode: 9521 Score: 154.0
Episode: 9522 Score: 48.0
Episode: 9523 Score: 54.0
Episode: 9524 Score: 50.0
Episode: 9525 Score: 85.0
Episode: 9526 Score: 71.0
Episode: 9527 Score: 83.0
Episode: 9528 Score: 65.0
Episode: 9529 Score: 45.0
Episode: 9530 Score: 129.0
Episode: 9531 Score: 49.0
Episode: 9532 Score: 143.0
Episode: 9533 Score: 39.0
Episode: 9534 Score: 77.0
Episode: 9535 Score: 67.0
Episode: 9536 Score: 38.0
Episode: 9537 Score: 90.0
Episode: 9538 Score: 120.0
Episode: 9539 Score: 56.0
Episode: 9540 Score: 72.0
Episode: 9541 Score: 45.0
Episode: 9542 Score: 40.0
Episode: 9543 Score: 71.0
Episode: 9544 Score: 36.0
Episode: 9545 Score: 48.0
Episode: 9546 Score: 37.0
Episode: 9547 Score: 36.0
Episode: 9548 Score: 73.0
Episode: 9549 Score: 100.0
Episode: 9550 Score: 43.0
Episode: 9551 Score: 58.0
Epis

Episode: 9847 Score: 81.0
Episode: 9848 Score: 29.0
Episode: 9849 Score: 59.0
Episode: 9850 Score: 166.0
Episode: 9851 Score: 63.0
Episode: 9852 Score: 67.0
Episode: 9853 Score: 95.0
Episode: 9854 Score: 204.0
Episode: 9855 Score: 103.0
Episode: 9856 Score: 69.0
Episode: 9857 Score: 46.0
Episode: 9858 Score: 95.0
Episode: 9859 Score: 167.0
Episode: 9860 Score: 59.0
Episode: 9861 Score: 40.0
Episode: 9862 Score: 267.0
Episode: 9863 Score: 59.0
Episode: 9864 Score: 40.0
Episode: 9865 Score: 84.0
Episode: 9866 Score: 60.0
Episode: 9867 Score: 85.0
Episode: 9868 Score: 35.0
Episode: 9869 Score: 88.0
Episode: 9870 Score: 59.0
Episode: 9871 Score: 51.0
Episode: 9872 Score: 87.0
Episode: 9873 Score: 43.0
Episode: 9874 Score: 71.0
Episode: 9875 Score: 39.0
Episode: 9876 Score: 56.0
Episode: 9877 Score: 45.0
Episode: 9878 Score: 55.0
Episode: 9879 Score: 83.0
Episode: 9880 Score: 111.0
Episode: 9881 Score: 100.0
Episode: 9882 Score: 59.0
Episode: 9883 Score: 46.0
Episode: 9884 Score: 41.0
Episo

In [7]:
state = env.reset()
state_size = 4
action_size = env.action_space.n
theta = np.random.random(state_size * action_size)

def _phi(s, a):
    encoded = np.zeros([action_size, state_size])
    encoded[a] = s
    return encoded.flatten()

def _softmax(s, a):
    return np.exp(theta.dot(_phi(s, a)) / 100)

def pi(s):
    """\pi(a | s)"""
    weights = np.empty(action_size)
    for a in range(action_size):
        weights[a] = _softmax(s, a)
    return weights / np.sum(weights)


def act(state):
    probs = pi(state)
    a = random.choices(range(0, action_size), weights=probs)
    a = a[0]
    pi = probs[a]
    return (a, pi)

In [8]:
theta

array([0.67059612, 0.25784005, 0.77806322, 0.75089143, 0.90915224,
       0.39152139, 0.26376316, 0.11291722])

In [9]:
action_size

2

In [10]:
np.zeros([action_size, state_size])

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [14]:
np.empty(action_size)

array([4.10666840e-06, 9.99995893e-01])

In [16]:
probs = pi(state)
a = random.choices(range(0, action_size), weights=probs)

In [17]:
probs

array([0.5000036, 0.4999964])

In [18]:
a

[0]